#### The work of this file is to generate the adversarial sample of the test data.

##### Training the defense model.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import logging
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
from art.attacks.evasion import DeepFool
from art.estimators.classification import KerasClassifier
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, CarliniL2Method, CarliniLInfMethod, ProjectedGradientDescent, DeepFool, ThresholdAttack, PixelAttack, SpatialTransformation, SquareAttack, ZooAttack, BoundaryAttack, HopSkipJump
from keras model import *
from input_data import *

dc = Data_Class()
dc.input_train_app()
dc.input_train_label()
dc.input_test_app()
dc.input_test_label()

learning_rate=0.05

input_shape=dc.train_app.shape[1:]
x_train=dc.train_app
y_train=dc.train_label
x_test=dc.test_app
y_test=dc.test_label

#The structure of the attack model.
model = cnn_model_defense(input_shape=input_shape)
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.SGD(lr=learning_rate),metrics=['accuracy'])


classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=50, batch_size=100)
classifier.save("../MNIST/adv/data/models/natural_defense_model.h5")

##### Building the generator of the adversarial example.

In [ ]:
from art.attacks.evasion import CarliniL2Method, SaliencyMapMethod, FastGradientMethod, ProjectedGradientDescent
crafter_cl = CarliniL2Method(classifier=classifier, max_iter=200) #CW2
crafter_jsma = SaliencyMapMethod(classifier=classifier) #JSMA
crafter_fgsm = FastGradientMethod(classifier) # eps = 0.3, bath_size = 32 #FGSM
crafter_pgd = ProjectedGradientDescent(estimator=classifier, norm=np.inf, eps=0.3, eps_step=0.01, batch_size=32) #PGD

##### Generating the adversarial example and measuring the time delay of the generation of the adversarial example.

In [ ]:
import time
# CW2
time_start_cw2 = time.time()
x_test_adv_cw2 = crafter_cl.generate(x_test)
time_end_cw2 = time.time()
average_time_cw2 = (time_end - time_start)/500
#JSMA
time_start_jsma = time.time()
x_test_adv_jsma = crafter_jsma.generate(x_test, A)
time_end_jsma = time.time()
average_time_jsma = (time_end_jsma - time_start_jsma)/500
#FGSM
time_start_fgsm = time.time()
x_test_adv_fgsm = crafter_fgsm.generate(x_test)
time_end_fgsm = time.time()
average_time_fgsm = (time_end_fgsm - time_start_fgsm)/500
#PGD
time_start_pgd = time.time()
x_test_adv_pgd = crafter_pgd.generate(x_test)
time_end_pgd = time.time()
average_time_pgd = (time_end_pgd - time_start_pgd)/500
#Save the adversarial example of the test data.
np.save("./x_test_adv_cw2.npy", x_test_adv_cw2)
np.save("./x_test_adv_jsma.npy", x_test_adv_jsma)
np.save("./x_test_adv_fgsm.npy", x_test_adv_fgsm)
np.save("./x_test_adv_pgd.npy", x_test_adv_pgd)
#Print the time delay generating the adversarial examples for different methods.
print(average_time_cw2)
print(average_time_jsma)
print(average_time_fgsm)
print(average_time_pgd)